In [1]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
import json

In [3]:
def get_water_usage(objId, cluster, client_id, client_secret, authority_id, monthly = True):
    kcsb = KustoConnectionStringBuilder.with_aad_application_key_authentication(cluster, client_id, client_secret, authority_id)
    client = KustoClient(kcsb)
    db = "unity-consumption"
    
    if monthly:
        query = "consumption | extend Month=format_datetime(['time'],\"MM\") | summarize TotalConsumed = sum(Value) by ManagedObjectid, Month| where ManagedObjectid =="+str(objId)
    else:
        query = "consumption | summarize TotalConsumed = sum(Value) by ManagedObjectid, bin(['time'], 7d) | where ManagedObjectid =="+str(objId)
    
    response = client.execute(db, query)
    json_result = json.loads(str(response.primary_results[0]))
    return json_result['data']